In [113]:
import random

first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]

# Set a seed for reproducibility
random.seed(42)

# Generate 24 random names
random_names = []
for _ in range(24):
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    random_names.append(f"{first_name} {last_name}")

# Display the generated names
for name in random_names:
    print(name)


Hamed Hosseini
Ali Niknam
Mohammad Rahimi
Sara Karimi
Roya Hosseini
Hamed Niknam
Mehdi Hosseini
Narges Moradi
Ali Ahmadi
Zahra Rahimi
Sara Rostami
Narges Ahmadi
Mehdi Rahimi
Roya Hosseinzadeh
Roya Rostami
Hossein Rahimi
Maryam Fazeli
Mohammad Ahmadi
Reza Niknam
Hossein Ebrahimi
Mohammad Karimi
Sara Ebrahimi
Zahra Hosseini
Hossein Hosseini


In [74]:
import numpy as np

# Generate a constraint matrix for 24 people
constraint_matrix = np.zeros((24, 24), dtype=int)

# Randomly select 40 pairs of individuals who should not sit next to each other
random.seed(42)
constraints = set()
while len(constraints) < 40:
    i, j = random.sample(range(24), 2)
    if (i, j) not in constraints and (j, i) not in constraints:
        constraints.add((i, j))
        constraint_matrix[i, j] = 1
        constraint_matrix[j, i] = 1

# Display the constraint pairs
for i, j in constraints:
    print(f"{random_names[i]} should not sit next to {random_names[j]}")

print("\nConstraint Matrix:")
print(constraint_matrix)

Zahra Hosseini should not sit next to Mohammad Karimi
Mohammad Ahmadi should not sit next to Sara Karimi
Hossein Ebrahimi should not sit next to Hamed Hosseini
Zahra Hosseini should not sit next to Mohammad Ahmadi
Ali Ahmadi should not sit next to Hamed Hosseini
Mohammad Ahmadi should not sit next to Mehdi Hosseini
Mohammad Ahmadi should not sit next to Zahra Rahimi
Zahra Rahimi should not sit next to Mohammad Rahimi
Narges Ahmadi should not sit next to Hamed Niknam
Ali Niknam should not sit next to Hamed Hosseini
Hamed Niknam should not sit next to Zahra Hosseini
Hamed Hosseini should not sit next to Hossein Hosseini
Narges Moradi should not sit next to Roya Hosseini
Narges Moradi should not sit next to Maryam Fazeli
Reza Niknam should not sit next to Roya Hosseinzadeh
Hossein Hosseini should not sit next to Sara Karimi
Mehdi Rahimi should not sit next to Sara Karimi
Mohammad Ahmadi should not sit next to Mohammad Rahimi
Mohammad Karimi should not sit next to Hossein Ebrahimi
Roya Ros

In [80]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
class SeatingModel(nn.Module):
    def __init__(self):
        super(SeatingModel, self).__init__()
        self.fc1 = nn.Linear(24, 128)
        self.fc2 = nn.Linear(128, 24)
      

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer

model = SeatingModel()

def loss(output, constraint_matrix):

    probabilities = torch.softmax(output, dim=1)
    probabilities = probabilities.reshape(24)

   
    penalty = 0
    for i in range(23):
        for j in range(i + 1, 23):
            if conflict_matrix[i, j] == 1:
                penalty += probabilities[i] * probabilities[j]


    return penalty

optimizer = optim.Adam(model.parameters())

In [110]:
import random
import torch

class SeatingArrangement:
    def __init__(self, n_people=24, rows=6, cols=4, constraint_matrix = constraint_matrix):
        
        self.n_people = n_people
        self.rows = rows
        self.cols = cols
        self.constraint_matrix = constraint_matrix

    def generate_seating_order(self):

        person_ids = [i for i in range(self.n_people)]
        random.shuffle(person_ids)

        seating_order = torch.zeros(self.rows, self.cols)
        k = 0
        for i in range(self.rows):
            for j in range(self.cols):
                seating_order[i, j] = person_ids[k]
                k += 1

        return seating_order.float()

    def analyze_seating_order_conflicts(self, seating_order, constraint_matrix):

        seating_order_conflicts = torch.zeros(self.rows, self.cols)

        # Iterate over the seating order and check for conflicts
        for i in range(self.rows):
            for j in range(self.cols):
                person = int(seating_order[i, j])

                # Check conflicts with adjacent persons in the same row
                if j > 0:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i, j - 1])]
                if j < self.cols - 1:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i, j + 1])]

                # Check conflicts with adjacent persons in the same column
                if i > 0:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i - 1, j])]
                if i < self.rows - 1:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i + 1, j])]

        return seating_order_conflicts

    def find_incompatible_pairs(self, seating_order_conflicts):

        incompatible_pairs = []
        for i in range(seating_order_conflicts.shape[0]):
            for j in range(seating_order_conflicts.shape[1]):
                if seating_order_conflicts[i, j] != 0:
                    incompatible_pairs.append((i, j))
        return incompatible_pairs

    def get_conflicted_seats(self, seating_order, incompatible_pairs):

        seat_conflict = []
        for i, j in incompatible_pairs:
            seat_conflict.append(int(seating_order[i][j]))
        return seat_conflict

    def swap_conflict_seats(self, incompatible_pairs, seat_conflict, seating_order):

        for coord, value in zip(incompatible_pairs, seat_conflict):
            i, j = coord
            seating_order[i, j] = value
        return seating_order

    def update_conflict_new_order(self, seating_order, constraint_matrix):

        update_matrix = self.analyze_seating_order_conflicts(seating_order, conflict_matrix)
        return update_matrix

In [112]:
# Training loop
rows, cols = 6, 4
loss_value = 0
seating_arrangement = SeatingArrangement(n_people=24, rows=6, cols=4, constraint_matrix=constraint_matrix)
seating_order = seating_arrangement.generate_seating_order()
seating_order_conflicts = seating_arrangement.analyze_seating_order_conflicts(seating_order, constraint_matrix)
print('seating_order:', seating_order)
print('seating_order_conflicts:', seating_order_conflicts)

# Number of epochs
n_epochs = 100
loss_values = []

for epoch in range(n_epochs):
    if epoch != 0:
        incompatible_pairs = seating_arrangement.find_incompatible_pairs(seating_order_conflicts)
        print('find incompatible pairs using conflict matrix:', incompatible_pairs)
        conflicted_seats = seating_arrangement.get_conflicted_seats(seating_order, incompatible_pairs)
        print('find incompatible seat numbers', conflicted_seats)

        if len(conflicted_seats) == 1:
            break

        random.shuffle(conflicted_seats)
        new_seating_order = seating_arrangement.swap_conflict_seats(incompatible_pairs, conflicted_seats, seating_order)
        print('new generated seating order:', new_seating_order)
        updated_conflict = seating_arrangement.update_conflict_new_order(seating_order, constraint_matrix)
        print('update conflict seating order:', updated_conflict)

        if len(conflicted_seats) == 2 and torch.sum(torch.tensor(updated_conflict) == 1) == 2:
            if loss_value < 1.000000e-06:
                break

        seating_order = new_seating_order
        seating_order_conflicts = updated_conflict

    # Predict seating probabilities
    output = model(torch.FloatTensor(seating_order_conflicts).view(-1, rows*cols))

    # Compute loss
    loss_value = loss(output, constraint_matrix)
    loss_values.append(loss_value.item())

    # Zero gradients, perform backward pass, and update weights
    optimizer.zero_grad()
    loss_value.backward()
    optimizer.step()

    # Display loss
    print(f"Epoch {epoch + 1}: loss = {loss_value.item():.7f}")

# Display all loss values
print("\nAll loss values:")
for epoch in range(1, n_epochs + 1):
    if epoch <= len(loss_values):
        print(f"Epoch {epoch}: loss = {loss_values[epoch - 1]:.7f}")
    else:
        print(f"Epoch {epoch}: loss = (not available)")

seating_order: tensor([[19.,  7.,  2.,  6.],
        [10., 21.,  4., 13.],
        [ 8., 23.,  0.,  5.],
        [ 1., 17., 15.,  3.],
        [18., 16., 14., 11.],
        [20., 12.,  9., 22.]])
seating_order_conflicts: tensor([[0.7629, 0.6394, 1.2213, 1.1279],
        [2.2279, 1.5738, 1.4034, 0.5501],
        [0.9669, 2.0334, 1.8010, 1.6559],
        [0.4096, 1.3911, 1.7797, 0.8134],
        [2.1122, 2.2616, 0.8882, 1.4804],
        [1.2676, 1.7669, 0.5822, 1.2264]])
Epoch 1: loss = 0.0668234
find incompatible pairs using conflict matrix: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3), (4, 0), (4, 1), (4, 2), (4, 3), (5, 0), (5, 1), (5, 2), (5, 3)]
find incompatible seat numbers [19, 7, 2, 6, 10, 21, 4, 13, 8, 23, 0, 5, 1, 17, 15, 3, 18, 16, 14, 11, 20, 12, 9, 22]
new generated seating order: tensor([[10.,  1., 19., 12.],
        [21., 17.,  4., 15.],
        [16., 20., 23.,  5.],
        [11., 14.,  6.,